In [ ]:
#!pip install gsutil

In [ ]:
#path_to_folder = ''
#path_data = '/'

In [ ]:
import pandas as pd
import PIL
from PIL import Image
from IPython.display import display
import os
import csv
from IPython.display import clear_output

In [ ]:
path_to_annotation_data = '/dollar_street_reannotated_with_income_v2.csv'

In [ ]:
data = pd.read_csv(path_to_annotation_data)
data.rename(columns={'identified object': 'identified_object'}, inplace=True)
data.head(1)

In [ ]:
data['topics'].nunique()

## Set up pipeline

In [ ]:
# filter
def contains_none(identified_object):
    return "None" in str(identified_object) or "no" in str(identified_object)

def filter(data, criteria = 'None'):
    '''criteria = ['None', 'Score', 'income']'''

    if criteria == 'None':
    df = data[data["identified_object"].apply(contains_none)]

    if criteria == 'Sentence':
    df = data[len(data["identified_object"].split(' ')) > 4]

    if criteria == 'Score':
    df = data[data["scores"] <= 0.2]

    if criteria == 'Income':
    df = data[data['income'] <= 500]

    return df


In [ ]:
#show image
path_prefix = 'gs://dollar-street/dataset_dollarstreet/'
import subprocess

def show_image(path_prefix, im_id):
    full_path = path_prefix + im_id
    print(full_path)
    #!gsutil cp gs://dollar-street/dataset_dollarstreet/assets/5d4bf101cf0b3a0f3f355d2c/5d4bf101cf0b3a0f3f355d2c.jpg /content/
    #command = f"gsutil gs://dollar-street/dataset_dollarstreet/assets/5d4bf101cf0b3a0f3f355d2c/5d4bf101cf0b3a0f3f355d2c.jpg /content/"
    !gsutil cp full_path /content/

    im_path = '/content/' + im_id.split('/')[-1]

    im = Image.open(im_path)
    #display(im)
    return

In [ ]:
#Update
def update(data, df, index, old_value, im_path):

    new_value = input(f"Previous annotation:{old_value}. What will you update the identified object cell with?: ")
    os.remove(im_path)

    data.loc[index, ['identified_object']] = new_value

    data.to_csv(path_to_folder + 'joan_nan.csv')
    return data, new_value

In [ ]:
#Track
def track_updates(track_list, criteria, index, old_value, new_value):
    mini_list = [criteria, index, old_value, new_value]
    track_list.append(mini_list)

    with open(path_to_folder +'track_list_2.csv', "w", newline="") as file:
    # Create a CSV writer object
    writer = csv.writer(file)
    # Write the list to the CSV file
    writer.writerows(track_list)

    clear_output(wait=True)
    return

In [ ]:
## Altogether

def show_current_values(row):

    old_value = row['identified_object']
    im_id = row['image ids']
    print(f"Index: {index}")
    print(f"Topic: {row['topics']}")
    print(f"Llava caption: {row['Llava captions']}")
    print(f"Function: {row['generated functions']}")
    print(f"Object: {row['identified_object']}")

    show_image(path_prefix, im_id)

    return old_value, im_id

## Iterative manual reannotation

In [ ]:
#initial run


track_list = []
path_to_folder = '/'
path_to_annotation_data = '/'
data = pd.read_csv(path_to_annotation_data)
data.rename(columns={'identified object': 'identified_object'}, inplace=True)
data.set_index('Unnamed: 0')

criteria = 'None'
df = data

df = filter(data, criteria)

for index, row in df.iterrows():
    old_value, im_id = show_current_values(row)

    im_path = '/content/' + im_id.split('/')[-1]
    data, new_value = update(data, df, index, old_value, im_path)
    track_updates(track_list, criteria, index, old_value, new_value)

In [ ]:
data.to_csv(path_to_folder + 'data/all_none_objects_addressed.csv')